In [1]:
import requests
import json
from IPython.core.display import display, HTML
import util

Set the drug that we want to find associations for

In [2]:
#myDrug = "INFLIXIMAB"
#myDrug = "GEFITINIB"
myDrug = "acetoxybenzoic acid"

Search for all diseases and targets associated with this drug

In [3]:
dataObject = util.drugSearch(myDrug)

Display the results

In [4]:
display(HTML(util.displayResults(myDrug, dataObject)))

Classification,Diseases,Targets
- Phase IV - Small molecule - http://identifiers.org/chembl.compound/CHEMBL509,- anxiety - arthralgia - Familial partial lipodystrophy - biliary liver cirrhosis - obesity - rheumatic disease - cirrhosis of liver,- GABRG3 - GABRG2 - ALOX5 - NR1H4 - GABRG1 - GABRA2


In [5]:
display(HTML(util.displayFoamTree(myDrug, dataObject)))


In [6]:
display(HTML(util.displayNetworkGraph()))

In [7]:
display(HTML(util.displayBubbleMenu(myDrug, dataObject)))